In [1]:
%load_ext dotenv
%dotenv
import os
from neo4j import GraphDatabase


AUTH = (os.getenv("NEO4J_USER"), os.getenv("NEO4J_PASSWORD"))

with GraphDatabase.driver(os.getenv("NEO4J_URL"), auth=(AUTH)) as driver:
    driver.verify_connectivity()

In [2]:
from importlib import reload
import openai_assistant

reload(openai_assistant)

<module 'openai_assistant' from '/Users/oskarhane/Development/tmp/neo4j-genai-design/openai_assistant.py'>

In [58]:
from custom_retrievers import movie
import retriever_router
import neo4j_retrievers
import neo4j_rag
reload(neo4j_retrievers)
reload(movie)
reload(retriever_router)
reload(neo4j_rag)

llm = openai_assistant.OpenAIAssitant()

movie_plot_similarity = neo4j_retrievers.VectorRetriever(
    driver=driver,
    llm=llm, 
    name="movie_plot_similarity",
    index_name="moviePlotsEmbedding"
)
text2cypher_retriever = neo4j_retrievers.Text2Cypher(driver=driver, llm=llm)
movie_retriever = movie.MovieRetriever(driver=driver)

agentic_retriever = retriever_router.RetrieverRouter(
    llm=llm, 
    retrievers=[movie_plot_similarity, movie_retriever, text2cypher_retriever]
)

rag_app = neo4j_rag.Neo4jRAG(llm=llm, retriever=agentic_retriever)
# or
single_retriever_rag_app = neo4j_rag.Neo4jRAG(llm=llm, retriever=text2cypher_retriever)


In [60]:
print("------ Agentic retriever ------")
response = rag_app.handle_input('What year was the movie "Casino" released?')
print("--- output ---")
print(response)

print("------ Single retriever ------")
response = single_retriever_rag_app.handle_input('What year was the movie "Casino" released?')
print("--- output ---")
print(response)

------ Agentic retriever ------
Expanded questions: ["What year was the movie 'Casino' released?"]
Updated question: When was the movie 'Casino' released?
MovieStrategy invoked: 'Casino' 'exact'
Response from tools: ['[{"movie": {"title": "Casino", "year": 1995, "plot": "Greed, deception, money, power, and murder occur between two mobster best friends and a trophy wife over a gambling empire."}, "actors": [{"name": "Robert De Niro", "role": "Sam \'Ace\' Rothstein"}, {"name": "Joe Pesci", "role": "Nicky Santoro"}, {"name": "Sharon Stone", "role": "Ginger McKenna"}, {"name": "James Woods", "role": "Lester Diamond"}]}]']
Unanswered questions: []
--- output ---
The movie "Casino" was released in 1995.
------ Single retriever ------
Text2Cypher invoked: What year was the movie "Casino" released?
MATCH (m:Movie {title: "Casino"}) RETURN m.year as year
--- output ---
The movie "Casino" was released in 1995.


In [45]:
response = rag_app.handle_input('What movie is about two mobster best friends, and what year was it released? And who were some actors playing roles in it?')
print("--- output ---")
print(response)


Expanded questions: ['What movie is about two mobster best friends?', 'When was the movie released?', 'Who were some actors playing roles in the movie?']
Updated question: What movie is about two mobster best friends and a trophy wife over a gambling empire?
VectorStrategy invoked: two mobster best friends and a trophy wife over a gambling empire
Response from tools: ['{"movie": {"title": "Casino", "year": 1995, "plot": "Greed, deception, money, power, and murder occur between two mobster best friends and a trophy wife over a gambling empire."}}']
Updated question: When was the movie 'Casino' released?
MovieStrategy invoked: 'Casino' 'exact'
Response from tools: ['[{"movie": {"title": "Casino", "year": 1995, "plot": "Greed, deception, money, power, and murder occur between two mobster best friends and a trophy wife over a gambling empire."}, "actors": [{"name": "Robert De Niro", "role": "Sam \'Ace\' Rothstein"}, {"name": "Joe Pesci", "role": "Nicky Santoro"}, {"name": "Sharon Stone", "

In [46]:
response = rag_app.handle_input('How many actors were in the movie "Casino"')
print("--- output ---")
print(response)

Expanded questions: ["How many actors were in the movie 'Casino'?"]
Updated question: How many actors played roles in the movie 'Casino'?
Text2Cypher invoked: How many actors played roles in the movie 'Casino'?
MATCH (a:Actor)-[r:ACTED_IN]->(m:Movie {title:'Casino'})
RETURN COUNT(DISTINCT a) AS actorCount;
Response from tools: ['{"query": "MATCH (a:Actor)-[r:ACTED_IN]->(m:Movie {title:\'Casino\'})\\nRETURN COUNT(DISTINCT a) AS actorCount;", "records": [{"actorCount": 4}]}']
Unanswered questions: []
--- output ---
According to the provided information, there were 4 actors in the movie "Casino."


In [50]:
response = rag_app.handle_input('What are two the highest rated movies Robert De Niro has acted in? What are the plots in those movies?')
print("--- output ---")
print(response)

Expanded questions: ['What are the two highest rated movies that Robert De Niro has acted in?', 'What is the plot of the first highest rated movie that Robert De Niro has acted in?', 'What is the plot of the second highest rated movie that Robert De Niro has acted in?']
Updated question: What are the titles of the two highest rated movies that Robert De Niro has acted in?
Text2Cypher invoked: What are the titles of the two highest rated movies that Robert De Niro has acted in?
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)
WHERE a.name = "Robert De Niro"
WITH m
ORDER BY m.imdbRating DESC
LIMIT 2
RETURN m.title

Response from tools: ['{"query": "MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)\\nWHERE a.name = \\"Robert De Niro\\"\\nWITH m\\nORDER BY m.imdbRating DESC\\nLIMIT 2\\nRETURN m.title\\n", "records": [{"m.title": "Godfather: Part II, The"}, {"m.title": "Goodfellas"}]}']
Updated question: What is the plot of the first highest rated movie that Robert De Niro has acted in?
MovieStrategy invoked: '